In [ ]:
Email
cat <<EOF > q2
#!/bin/bash
echo \$PWD
echo $PWD
EOF
#mail -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < test.txt  # working
test_file="/axp/buanalytics/csramp/dev/cor/data/pps_defect_spain_10152018.xlsx"
#mail -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  sophia.yue@aexp.com < test.txt  # send me twice ; the first sophia is sender the second and the third sophia are the receiptent
#mail -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com   < ${test_file} # attach test.txt
mail -s "test email" -a ${test_file} -r Sophia.yue@aexp.com sophia.yue@aexp.com < test.txt
#cat q2 | mail -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < test.txt

#cat q2 | mailx -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < test.txt
#cat q2 | mailx -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < body.txt
#cat q2 | mailx -s "test email" -a test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < test.txt  # not take q2 as body; take test.txt as body

#mail -s "test email" -a  test.txt -r Sophia.yue@aexp.com Sophia.yue@aexp.com  < test.txt
echo "send email"

/axp/platform/cloak/bin/spark/cloak-sparksubmit
 
#!/bin/bash

export SPARK_HOME=/opt/mapr/spark/spark-1.6.1/

/opt/mapr/spark/spark-1.6.1/bin/spark-submit --master yarn-client --driver-memory=16G --executor-memory=4G --conf spark.yarn.executor.memoryOverhead=2048 --conf spark.yarn.jar=maprfs:///axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar --conf spark.sql.parquet.binaryAsString=true --conf "spark.executor.extraJavaOptions=-XX:MaxDirectMemorySize=512m -XX:+UseConcMarkSweepGC -XX:+CMSParallelRemarkEnabled -XX:+UseCMSInitiatingOccupancyOnly -XX:CMSInitiatingOccupancyFraction=30 -XX:+ScavengeBeforeFullGC -XX:+CMSScavengeBeforeRemark" --conf "spark.driver.extraJavaOptions=-XX:MaxDirectMemorySize=512m -XX:+UseConcMarkSweepGC -XX:+CMSParallelRemarkEnabled -XX:+UseCMSInitiatingOccupancyOnly -XX:CMSInitiatingOccupancyFraction=30 -XX:+ScavengeBeforeFullGC -XX:+CMSScavengeBeforeRemark" --conf spark.network.timeout=240s --conf spark.akka.frameSize=1024 --conf spark.rpc.numRetries=5 --conf spark.speculation=true --conf spark.locality.wait=1s --conf spark.akka.threads=16 --conf spark.yarn.jar=maprfs:///axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar --jars /axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar --driver-class-path /axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar "$@"
cor_com_code_parm.py
 
import sys
import pandas as pd
from datetime import datetime, date
# Module name: tst_arg.py

# Without enter parm will use default setting to point to production folder and schema
run_type = 'prod'
yymm = (pd.Period(datetime.today(), 'M') - 1).strftime('%y%m')
path_code = "/axp/buanalytics/csramp/dev/cor/code_prod/"
path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
sch_nm = "pcr_cor"
print ("argument no = ", len(sys.argv))
if len(sys.argv) >= 2:
   run_type =  sys.argv[1].lower()

if  run_type.lower() == 'dev':
   path_code = "/axp/buanalytics/csramp/dev/cor/code/"
    path_data = "/axp/buanalytics/csramp/dev/cor/data/"
    sch_nm = "yuedb"


if len(sys.argv) >= 3:
   yymm =  sys.argv[2]

print(" run_type" , run_type, 'yymm', yymm, '\n', 'path_code', path_code, '\n', 'path_data', path_data)

cor_com_code_init.py
 
# load code to set up environmental variables to run SqlSpark/pyspark
f_set_spark = path_code + "cor_com_code_set_spark_sql.py"
exec(compile(open(f_set_spark , "rb").read(), f_set_spark, 'exec' )) # use exec to include the common files

# load code to import non pyspark/sqlspark related package/libraries
f_import = path_code + "cor_com_code_import.py"
exec(compile(open(f_import, "rb").read(), f_import, 'exec' )) # use exec to include the common files

# load functions
f_func = path_code + "cor_com_func.py"
exec(compile(open(f_func, "rb").read(), f_func, 'exec' )) # use exec to include the common files

cor_com_code_set_spark_sql.py
 
import os
os.environ['SPARK_HOME']='/opt/mapr/spark/spark/'
os.environ['PYSPARK_PYTHON']='/opt/python/python27/bin/python'

#import findspark
#findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, DataFrame
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, coalesce, col, concat
from pyspark.sql.types import *

conf = SparkConf()\
.setMaster("yarn-client")\
.setSparkHome('/opt/mapr/spark/spark/')\
.set('spark.executor.memory', '8g')\
.set('spark.driver.memory', '8g')\
.set('spark.yarn.jar','maprfs:///axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar')\
.setAll([('spark.io.compression.codec', 'lz4'), ('spark.sql.parquet.binaryasstring', 'true')])\
.set("spark.driver.extraClassPath","/axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar")\
.set('spark.shuffle.service.enabled', 'true') \
.set('spark.dynamicAllocation.enabled', 'true') \
.set("spark.dynamicAllocation.minExecutors", "10") \
.set("spark.dynamicAllocation.maxExecutors", "Integer.MAX_VALUE") \
.set("spark.dynamicAllocation.initialExecutors", "20") \
.set("spark.executor.instances", "20") \
.set("spark.jars","/axp/platform/cloak/lib/cloak-spark-1.2.0-SNAPSHOT.jar,/axp/platform/mlplat/app/lib/thirdpartyjars/spark-libs/commons-csv-1.4.jar,/axp/platform/mlplat/app/lib/thirdpartyjars/spark-libs/spark-csv_2.10-1.5.0.jar")
sc = SparkContext(conf=conf)
sqlContext = HiveContext(sc)

cor_com_code_defect_tbl_sch.py
 
from pyspark.sql.types import *
field = [StructField("toc_se10"              ,StringType(),  True),\
         StructField("se10"                  ,StringType(),  True),\
         StructField("se_lgl_ent_type_cd"    ,StringType(),  True),\
         StructField("ent_grp"               ,IntegerType(), True),\
         StructField("ent_cd_dsc"            ,StringType(),  True),\
         StructField("rol_typ_cd"            ,StringType(),  True),\
         StructField("rol_typ_grp"           ,StringType(),  True),\
         StructField("rol_typ_cd_dsc"        ,StringType(),  True),\
         StructField("mis_sumy"              ,StringType(),  True),\
         StructField("mis_sumy_det"          ,StringType(),  True)\
         ]
schema = StructType(field)

cor_com_code_import.py
 
import pandas as pd
import numpy as np
import os, time, re, gzip, sys, calendar

from datetime import datetime, date
from functools import reduce

from pandas import ExcelWriter
from pandas import ExcelFile

cor_ext_bld_bus_ctc_tbl.py
 
pgm = "cor_ext_bld_bus_ctc_tbl.py"
print ("start of " + pgm )
# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))


#load codefor Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

job_start_time = time.time()

#dedup

bus_all_tbl     = sch_nm + ".cor_wrk_bus_all"
q_rmv_fer= "select * from {0} where trim(ctc_nm) not in ('FFR POLICY AUTOMATION')".format(bus_all_tbl)
print("q_rmv_fer", q_rmv_fer)
sqlContext.sql(q_rmv_fer).registerTempTable('tmp_bus_ctc_tbl')

bus_dedup_a_tbl = sch_nm + ".cor_wrk_bus_dedup_a"
q_drop = 'drop table {0}'.format(bus_dedup_a_tbl)
sqlContext.sql(q_drop)
cr_cor_wrk_bus_dedup_a = """
        create table   {0} as
        select  * from
       (select bus.*,            
               row_number() over (partition by  ctry_cd, toc_se10, se10,
                                                se_lgl_ent_type_cd, first_rspbl_cd, scnd_rspbl_cd, third_rspbl_cd, fourth_rspbl_cd, fifth_rspbl_cd                                               
                                      order  by ctry_cd, toc_se10, se10,
                                                se_lgl_ent_type_cd, first_rspbl_cd, scnd_rspbl_cd, third_rspbl_cd, fourth_rspbl_cd, fifth_rspbl_cd,                                              
                                                cstone_last_updatetm  desc, end_dt desc) as seqnum
      from   tmp_bus_ctc_tbl  bus
      ) tmp
      where seqnum = 1""".format(bus_dedup_a_tbl)
print("SQL for dedupping", cr_cor_wrk_bus_dedup_a)
sqlContext.sql(cr_cor_wrk_bus_dedup_a )
f_get_tbl_cnt(bus_dedup_a_tbl, '')


#eliminate inactive SE10
bus_dedup_tbl = sch_nm + ".cor_wrk_bus_dedup"
q_drop = 'drop table {0}'.format(bus_dedup_tbl)
sqlContext.sql(q_drop)
cr_cor_wrk_bus_dedup ="""
        create table {0}  as
        select   regexp_replace(abbr_nm,' ','_') as abbr_nm, bus.*  from {1} bus
        left outer join cstonedb3.crt_country cty
        on    bus.ctry_cd  = cty.ctry_id
        where bus.end_dt = '9999-12-31' """.format(bus_dedup_tbl, bus_dedup_a_tbl)
print("SQL to eliminate inactive SE10: ", cr_cor_wrk_bus_dedup)
sqlContext.sql(cr_cor_wrk_bus_dedup )
f_get_tbl_cnt(bus_dedup_tbl, '')


#add entity group & role type


rol_tbl  = sch_nm + ".cor_lup_bus_role"
ent_tbl  = sch_nm + ".cor_lup_leg_ent"

bus_ctc_pre_tbl = sch_nm + ".cor_wrk_bus_ctc_pre"
q_drop = 'drop table {0}'.format(bus_ctc_pre_tbl)
sqlContext.sql(q_drop)
q_cr_bus_ctc = """
        create table {0} as
        select ent.ent_grp,
               rol1.bus_rol_grp_abb as bus_rol_grp_abb_1,
               rol2.bus_rol_grp_abb as bus_rol_grp_abb_2,                  
               rol3.bus_rol_grp_abb as bus_rol_grp_abb_3,
               rol4.bus_rol_grp_abb as bus_rol_grp_abb_4,
               rol5.bus_rol_grp_abb as bus_rol_grp_abb_5,              
               bus.*
        from {1}  bus        
        left outer join {2} rol1  
        on   trim(first_rspbl_cd) = rol1.bus_rol_cd
        
        left outer join {3} rol2  
        on   trim(scnd_rspbl_cd) = rol2.bus_rol_cd  
        
        left outer join {4} rol3  
        on   trim(third_rspbl_cd) = rol3.bus_rol_cd
        
        left outer join {5} rol4  
        on   trim(fourth_rspbl_cd) = rol4.bus_rol_cd
        
        left outer join {6} rol5  
        on   trim(fifth_rspbl_cd) = rol5.bus_rol_cd
        
        left outer join {7} ent
        on  (lcase(bus.abbr_nm) = lcase(ent.cty)
        and  trim(se_lgl_ent_type_cd) =  ent.ent_cd)        
    """.format(bus_ctc_pre_tbl, bus_dedup_tbl, rol_tbl, rol_tbl, rol_tbl, rol_tbl, rol_tbl, ent_tbl )
print("SQL to entity group & role type: ", q_cr_bus_ctc)
sqlContext.sql( q_cr_bus_ctc )
f_get_tbl_cnt(bus_ctc_pre_tbl, '')

#add rol_typ
bus_ctc_tbl = sch_nm + ".cor_wrk_bus_ctc"
q_drop = 'drop table {0}'.format(bus_ctc_tbl)
sqlContext.sql(q_drop)

q_cr_bus_ctc = """
        create table {0}  as
         select *,  
                concat( coalesce(bus_rol_grp_abb_1,""), coalesce(bus_rol_grp_abb_2,""),  coalesce(bus_rol_grp_abb_3,""),  coalesce(bus_rol_grp_abb_4,""),  coalesce(bus_rol_grp_abb_5,"") ) as rol_typ
         from  {1}  sort by ctry_cd, se10
       
         """.format(bus_ctc_tbl, bus_ctc_pre_tbl)
print("Sql to add rol_typ: ", q_cr_bus_ctc)
sqlContext.sql( q_cr_bus_ctc )

job_end_time = time.time()

elapse_time (job_start_time, job_end_time, pgm)
print ("end of " + pgm )

cor_ext_bld_mernm_stscd_tbl.py
gm = "cor_ext_bld_mernm_stscd_tbl.py"

# Invoke code to get parm to determine the location of the folder and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

job_start_time = time.time()
end_setup_date = f_dte_for_yymm(yymm, 'l')  # get the last day of yymm
start_setup_date = '2018-10-12'
print ("start_setup_date is", start_setup_date , "end_setup_date is", end_setup_date  )

#create the driver table

mernm_tbl  =  sch_nm + ".cor_wrk_mernm_stscd"
q_drop = 'drop table if exists {0}'.format(mernm_tbl)
sqlContext.sql(q_drop)
q_cr_mernm = """create table {0} as
                select mer.se10, mer.se_dba_nm, mer.se_sta_cd,  mer.se_sta_cd_ds
                  from cstonedb3.gms_merchant_char mer,
                       cstonedb3.gmdl_mer_spc spc
                 where spc.spcl_prog_cd in ('002','020')
                   and trim(spc.se10) = trim(mer.se10)
                   and mer.se_setup_dt between  '{1}' and '{2}'
                 group by mer.se10, mer.se_dba_nm, mer.se_sta_cd,  mer.se_sta_cd_ds
            """.format(mernm_tbl, start_setup_date, end_setup_date )
print("SQL to create the driver table : ", q_cr_mernm)
sqlContext.sql(q_cr_mernm)
f_get_tbl_cnt(mernm_tbl, '')

job_end_time = time.time()

elapse_time (job_start_time, job_end_time, pgm)
print ("end of " + pgm )


 
##  t_nw_bus_role.py
 
# From Rajan RAJAN CHATTERJEE <RAJAN.CHATTERJEE@aexp.com> and it is not working
print ("Start of cor_val_bus_type.py")

################################################################################################################################################
#ini - Initialization
################################################################################################################################################

# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load codefor Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

job_start_time = time.time()
#load code to define the schema for defect table
f_defect_tbl = path_code + "cor_com_code_defect_tbl_sch.py"
exec(compile(open(f_defect_tbl, "rb").read(), f_defect_tbl, 'exec' ))


#build key dictionary for group role type description, the key is the abbrevaitionn of role group
d_bus_rol_grp_dsc = { 'S': 'Signer',
                   'B': 'Beneficial Owner',
                   'D': 'Director',
                   'A': 'Authorized Contact'}

f_lup  = path_data + "cor_lup_leg_ent.csv"
df_lup = pd.read_csv(f_lup)
l_val = df_lup['ent_cd_dsc'].tolist()
l_key = df_lup['ent_cd'].tolist()
d_ent_dsc = dict (zip(l_key, l_val))

#ini3 - dynamically create the variables for required business role group
dsc = "Dynamically create the variables for required business role group."
start_time = time.time()    # need to import time
bus_rule_tbl = sch_nm + ".cor_lup_bus_rule"
q_cty = "select cty from {0}  group by cty".format(bus_rule_tbl)
_l_cty = sqlContext.sql(q_cty).collect()
for i in range(len(_l_cty)):
    cty = _l_cty[i]["cty"]
    q_ent_grp = 'select ent_grp from {0} where  cty= "{1}"  group by ent_grp sort by ent_grp'.format(bus_rule_tbl, cty)
    _l_ent_grp = sqlContext.sql(q_ent_grp).collect()
    for i  in range(len(_l_ent_grp)):
        ent_grp = _l_ent_grp[i]["ent_grp"]
        #print ('ent_grp', ent_grp)

        q_rol_typ = 'select ent_grp, rol_typ from {0} where  cty= "{1}" and ent_grp = {2} and req_cd = "M"  '.format(bus_rule_tbl, cty, ent_grp)
        #print ("q_rol_typ", q_rol_typ)
        df_q = sqlContext.sql(q_rol_typ)
        #df_q = sqlContext.sql("select ent_grp, rol_typ_desc, rol_typ from yuedb.cor_lup_bus_rule  req_cd=1 and cty='UK'")
        cty_ent_grp = cty.lower() + str(ent_grp)
        # globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp ) ] = df_q.groupby("ent_grp").agg(F.collect_set("rol_typ")).collect ()
        l_rol_typ =  df_q.groupby("ent_grp").agg(F.collect_set("rol_typ")).collect()
        globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp ) ] =  l_rol_typ [0]["collect_set(rol_typ)"]
end_time = time.time()  # need to import time
elapse_time (start_time, end_time, dsc)

################################################################################################################################################
#Main process starts here
################################################################################################################################################

#yymm=str(datetime.today().strftime('%y%m'))
bus_ctc_tbl = sch_nm + ".cor_bus_ctc_" + yymm

se10 = '9999999999'
q_sdf = 'select toc_se10, se_lgl_ent_type_cd, se10, ent_grp, rol_typ, abbr_nm from {0} where  se10 <= "{1}" '.format(bus_ctc_tbl, se10)

sdf = sqlContext.sql(q_sdf).toPandas()

start_time_all_cty = time.time()
_l_cty = sdf.abbr_nm.unique().tolist()
_l_mis_ary=[]
for cty in _l_cty:
    start_time = time.time()

    sc_ary =sdf.loc[sdf.abbr_nm==cty]["se10"].unique().tolist()
    dsc_cty =  str(len(sc_ary)) + " se10 had been validated for " + cty + "."
    print ("Start to validate missing business role for " + cty + " with " + str(len(sc_ary)) + " se10. ")
    sdf_rol = sdf[["se10","ent_grp","rol_typ"]].groupby('se10').agg({'rol_typ':lambda x: list(x), 'ent_grp':lambda y: list(y)[0]})
    for _se10 in sc_ary:

        l_rol_typ=sdf_rol[sdf_rol.index==str(_se10).strip()]['rol_typ'].tolist()[0]
        ent_grp =  sdf_rol[sdf_rol.index==str(_se10).strip()]['ent_grp'].tolist()[0]
        cty_ent_grp= cty.lower() + str(ent_grp)
        req_rol_typ_ent = globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp) ]
        mis_rol_typ = list(set(req_rol_typ_ent) - set(l_rol_typ ))  #val1.5 - validate if role_typwe is missing
        if len(mis_rol_typ) > 0:
             _toc_se10 = sdf.loc[sdf.se10==str(_se10).strip()]['toc_se10'].values[0]
             _se_lgl_ent_type_cd= sdf.loc[sdf.se10==str(_se10).strip()]['se_lgl_ent_type_cd'].values[0]
             _ent_cd_dsc = d_ent_dsc[int(_se_lgl_ent_type_cd)] + "- " + _se_lgl_ent_type_cd
             _rol_typ_grp    = "   "
             _rol_typ_cd     = "   "
             _rol_typ_cd_dsc = "   "
             l_mis_rol_typ_dsc  = list(map(lambda x: d_bus_rol_grp_dsc[x], mis_rol_typ))   # convert rol_typ abbreviation to description. eg, ['S', 'D'] would be ['Signer', 'Director']
             mis_sumy_det  = "Missing Role : " + ",".join(l_mis_rol_typ_dsc)
             
             mis_sumy      = ""
             _l_mis_ary.append((_toc_se10, _se10, _se_lgl_ent_type_cd, ent_grp, _ent_cd_dsc, _rol_typ_cd, _rol_typ_grp, _rol_typ_cd_dsc,  mis_sumy, mis_sumy_det ))
 
    end_time = time.time()  
    elapse_time (start_time, end_time, dsc_cty)
print("_l_mis_ary",_l_mis_ary[0:10])
len(sc_ary)
dsc_all_cty = "Validate business role for all countries with " + str ( len(_l_mis_ary)) + " defect records."
end_time_all_cty = time.time()  # need to import time
elapse_time (start_time_all_cty, end_time_all_cty, dsc_all_cty)       

df_defect = sqlContext.createDataFrame(sc.parallelize(_l_mis_ary), schema)
sqlContext.registerDataFrameAsTable(df_defect, "cor_defect")

defect_tbl= sch_nm + ".cor_wrk_defect_bus_rolex_" + yymm

q_drop = "drop table if exists {0}".format(defect_tbl)
sqlContext.sql(q_drop)
q_cr_tbl = "create table {0} as select * from cor_defect".format(defect_tbl)
sqlContext.sql(q_cr_tbl)
job_end_time = time.time()         
job_dsc = "Validate business role."
elapse_time (job_start_time, job_end_time, job_dsc)
print ("end of cor_val_bus_type.py")        
cor_val_bus_role.py
 
# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))


#load codefor Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

job_start_time = time.time()
#load code to define the schema for defect table
f_defect_tbl = path_code + "cor_com_code_defect_tbl_sch.py"
exec(compile(open(f_defect_tbl, "rb").read(), f_defect_tbl, 'exec' ))


#build key dictionary for group role type description, the key is the abbrevaitionn of role group
d_bus_rol_grp_dsc = { 'S': 'Signer',
                   'B': 'Beneficial Owner',
                   'D': 'Director',
                   'A': 'Authorized Contact'}

f_lup  = path_data + "cor_lup_leg_ent.csv"
df_lup = pd.read_csv(f_lup)
l_val = df_lup['ent_cd_dsc'].tolist()
l_key = df_lup['ent_cd'].tolist()
d_ent_dsc = dict (zip(l_key, l_val))

#ini3 - dynamically create the variables for required business role group
dsc = "Dynamically create the variables for required business role group."
start_time = time.time()    # need to import time
bus_rule_tbl = sch_nm + ".cor_lup_bus_rule"
q_cty = "select cty from {0}  group by cty".format(bus_rule_tbl)
_l_cty = sqlContext.sql(q_cty).collect()
for i in range(len(_l_cty)):
    cty = _l_cty[i]["cty"]
    q_ent_grp = 'select ent_grp from {0} where  cty= "{1}"  group by ent_grp sort by ent_grp'.format(bus_rule_tbl, cty)
    _l_ent_grp = sqlContext.sql(q_ent_grp).collect()
    for i  in range(len(_l_ent_grp)):
        ent_grp = _l_ent_grp[i]["ent_grp"]
        #print ('ent_grp', ent_grp)

        q_rol_typ = 'select ent_grp, rol_typ from {0} where  cty= "{1}" and ent_grp = {2} and req_cd = "M"  '.format(bus_rule_tbl, cty, ent_grp)
        #print ("q_rol_typ", q_rol_typ)
        df_q = sqlContext.sql(q_rol_typ)
        #df_q = sqlContext.sql("select ent_grp, rol_typ_desc, rol_typ from yuedb.cor_lup_bus_rule  req_cd=1 and cty='UK'")
        cty_ent_grp = cty.lower() + str(ent_grp)
        # globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp ) ] = df_q.groupby("ent_grp").agg(F.collect_set("rol_typ")).collect ()
        l_rol_typ =  df_q.groupby("ent_grp").agg(F.collect_set("rol_typ")).collect()
        globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp ) ] =  l_rol_typ [0]["collect_set(rol_typ)"]
end_time = time.time()  # need to import time
elapse_time (start_time, end_time, dsc)
         
################################################################################################################################################
#  Main process starts here
################################################################################################################################################

bus_ctc_tbl = sch_nm + ".cor_bus_ctc_" + yymm
#bus_ctc_tbl = "yuedb.cor_bus_ctc_" + yymm   # table with index & rol_typ

se10 = '9999999999'
_sdf = 'select toc_se10, se10, ent_grp, rol_typ, abbr_nm, se_lgl_ent_type_cd from {0} where se10 <= "{1}" '.format(bus_ctc_tbl, se10)
#q_sdf = 'select toc_se10, se10, ent_grp, rol_typ, abbr_nm, se_lgl_ent_type_cd from {0} where abbr_nm = "SPAIN" and se10 <= "{1}" '.format(bus_ctc_tbl, se10)

sdf = sqlContext.sql(q_sdf).cache()  # cache doesn't benifit the performance
#sdf = sqlContext.sql(q_sdf)


start_time_all_cty = time.time()
_l_cty = sdf.select("abbr_nm" ).distinct().collect()  
_l_mis_ary=[]
for _cty in _l_cty:
    cty  = _cty.__getitem__("abbr_nm")  
    start_time = time.time()

    sc_ary =sdf.where(sdf.abbr_nm  == cty).select("se10").distinct().collect()
    dsc_cty =  str(len(sc_ary)) + " se10 had been validated for " + cty + "."
                   
    for _sc in sc_ary:  
        _se10 = _sc.__getitem__("se10")
        _l_rol_typ = sdf.where(sdf.se10 == _se10).select(["ent_grp", "rol_typ", "toc_se10", "se_lgl_ent_type_cd" ]).collect()         
        l_rol_typ=[] #val1.3  - Create a List all the role types for the SE from the the query result
    
        for _rol_typ  in _l_rol_typ:
            rol_typ = list(_rol_typ.__getitem__("rol_typ"))
            l_rol_typ.append(rol_typ)
            
        l_rol_typ = sum(l_rol_typ, [])    #val1.4 -  Create Flat Lists Out Of Lists
        ent_grp =  _l_rol_typ[0]["ent_grp"]
        cty_ent_grp= cty.lower() + str(ent_grp)
        req_rol_typ_ent = globals()['req_rol_typ_ent_{0}'.format(cty_ent_grp) ]
        mis_rol_typ = list(set(req_rol_typ_ent) - set(l_rol_typ ))  #val1.5 - validate if role_typwe is missing
        if len(mis_rol_typ) > 0:
             _toc_se10            = _l_rol_typ[0]["toc_se10"]      
             _se_lgl_ent_type_cd  = _l_rol_typ[0]["se_lgl_ent_type_cd"]    
             _ent_cd_dsc     = d_ent_dsc[int(_se_lgl_ent_type_cd)] + "- " + _se_lgl_ent_type_cd
             _rol_typ_grp    = "   "
             _rol_typ_cd     = "   "
             _rol_typ_cd_dsc = "   "
             l_mis_rol_typ_dsc  = list(map(lambda x: d_bus_rol_grp_dsc[x], mis_rol_typ))   # convert rol_typ abbreviation to description. eg, ['S', 'D'] would be ['Signer', 'Director']
             mis_sumy_det  = "Missing Role : " + ",".join(l_mis_rol_typ_dsc)
             
             mis_sumy      = ""
             _l_mis_ary.append((_toc_se10, _se10, _se_lgl_ent_type_cd, ent_grp, _ent_cd_dsc, _rol_typ_cd, _rol_typ_grp, _rol_typ_cd_dsc,  mis_sumy, mis_sumy_det ))
 
    end_time = time.time()  
    elapse_time (start_time, end_time, dsc_cty)
print("_l_mis_ary",_l_mis_ary[0:10])
len(sc_ary)
dsc_all_cty = "Validate business role for all countries with " + str ( len(_l_mis_ary)) + " defect records."
end_time_all_cty = time.time()  # need to import time
elapse_time (start_time_all_cty, end_time_all_cty, dsc_all_cty)         

df_defect = sqlContext.createDataFrame(sc.parallelize(_l_mis_ary), schema)
sqlContext.registerDataFrameAsTable(df_defect, "cor_defect")

defect_tbl= sch_nm + ".cor_wrk_defect_bus_role_" + yymm

q_drop = "drop table if exists {0}".format(defect_tbl)
sqlContext.sql(q_drop)
q_cr_tbl = "create table {0} as select * from cor_defect".format(defect_tbl)
sqlContext.sql(q_cr_tbl)
f_get_tbl_cnt(defect_tbl, "")
job_end_time = time.time()         
job_dsc = "Validate business role."
elapse_time (job_start_time, job_end_time, job_dsc)
print ("end of cor_val_bus_type.py")      

cor_val_bus_data.py
pgm = "cor_val_bus_data.py"
print ("Start of " + pgm)
################################################################################################################################################
#ini - Initialization
################################################################################################################################################

# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))
job_start_time = time.time()

dsc = "Initialization."
start_time = time.time()    # need to import time


def f_isnull(value):
    if value: return False
    else: return True

#load code to define the schema for defect table
f_defect_tbl = path_code + "cor_com_code_defect_tbl_sch.py"
exec(compile(open(f_defect_tbl, "rb").read(), f_defect_tbl, 'exec' ))

#build key dictionary for group role type description, the key is the abbrevaitionn of role group
d_bus_rol_grp_dsc = { 'S': 'Signer',
                   'B': 'Beneficial Owner',
                   'D': 'Director',
                   'A': 'Authorized Contact'}

f_lup  = path_data + "cor_lup_leg_ent.csv"
df_lup = pd.read_csv(f_lup)
l_val = df_lup['ent_cd_dsc'].tolist()
l_key = df_lup['ent_cd'].tolist()
d_ent_dsc = dict (zip(l_key, l_val))

f_lup  = path_data + "cor_lup_bus_role.csv"
df_lup = pd.read_csv(f_lup)
l_val  = df_lup['bus_rol_dsc'].tolist()
l_key  = df_lup['bus_rol_cd'].tolist()
d_bus_rol_dsc  = dict (zip(l_key, l_val))

f_lup  = path_data + "cor_lup_corns_fld.csv"
df_lup = pd.read_csv(f_lup)
l_val = df_lup['bus_fld_nam'].tolist()
l_key = df_lup['corns_fld_nam'].tolist()
d_fld_nam  = dict (zip(l_key, l_val))
#-----------------------------------------------------------------------------------------------------------------------------------------------
#ini - Get country name  
#-----------------------------------------------------------------------------------------------------------------------------------------------
_bus_ctc_tbl = sch_nm + ".cor_bus_ctc_" + yymm
f_get_tbl_cnt(_bus_ctc_tbl, "")

bus_ctc_tbl = sqlContext.table(_bus_ctc_tbl).select("abbr_nm").toPandas()
_l_ctry_nm = bus_ctc_tbl.abbr_nm.unique().tolist()   # will be used by loop later e.g. ['AUSTRALIA', 'DENMARK', 'NORWAY', 'SPAIN', 'UK'] ;
#_l_ctry_nm = [str.encode('ascii') for str in _l_ctry_nm]   # convert from unicode  to ascii; having 'unicode' object is not callable for the following code
#s_ctry_nm_in = str(tuple(_l_ctry_nm))  # convert the list to string and will feed qry e.g "('AUSTRALIA', 'DENMARK', 'NORWAY', 'SPAIN', 'UK')"
s_ctry_nm_in = str(tuple(_l_ctry_nm)).replace("u'", "'").replace(',)', ')')  # convert the list to string and will feed qry e.g "('AUSTRALIA', 'DENMARK', 'NORWAY', 'SPAIN', 'UK')"

#-----------------------------------------------------------------------------------------------------------------------------------------------
#ini - dynamically create variables with suffix as rule id. e.g. if rule_id =  uk2a  
#      
#      - val_s_var_fld_uk2a   : a string with required fields for validation  .e.g. 'CTC_NM, BIRTH_DT, RESIDE_AD_LINE1, .... RESIDE_CTRY_CD'
#      - val_no_var_fld_uk2a : no of required fields
#-----------------------------------------------------------------------------------------------------------------------------------------------

_s_req_fld = ""
NoneType = type(None)
bus_rule_tbl = sch_nm + ".cor_lup_bus_rule"
q_bus_rule = "select *  from {0} where cty in {1} ".format(bus_rule_tbl, s_ctry_nm_in)
#print("q_bus_rule ", q_bus_rule, "\n", "s_ctry_nm_in" , s_ctry_nm_in , '\n', "_l_ctry_nm", _l_ctry_nm, "tuple(_l_ctry_nm",  tuple(_l_ctry_nm ))
bus_rul = sqlContext.sql(q_bus_rule ).collect()
for i in range(len(bus_rul)):

    rul_id = bus_rul[i]["rul_id"].lower()      # get the rule id
    s_req_fld = bus_rul[i]["req_fld"]        # return a string with variable(s) specified in reg_fld. e.g. 'CTC_NM  BIRTH_DT  RESIDE_AD_LINE1 .... RESIDE_CTRY_CD'
    globals()['val_req_cd_{0}'.format( rul_id)] =  bus_rul[i]["req_cd"].lower()
 
    # return a list of fields required for validation  
    globals()['val_no_var_fld_{0}'.format( rul_id) ] = 0
    if type(s_req_fld) != NoneType:
       _s_req_fld = _s_req_fld + " " + s_req_fld   # type = string       
       l_req_fld = s_req_fld.split()             # convert the string to a list, e.g.   ['CTC_NM', 'BIRTH_DT', 'RESIDE_AD_LINE1', ....  'RESIDE_CTRY_CD']
       globals()['val_no_var_fld_{0}'.format( rul_id)]  =  len(l_req_fld)

       s_req_fld = re.sub(' +', ' ', s_req_fld.strip() ).replace(" ", ", ") # change duplicate spaces to a single space and replace space to a comma e.g. 'CTC_NM, BIRTH_DT, RESIDE_AD_LINE1, .... RESIDE_CTRY_CD'
       globals()['val_s_var_fld_{0}'.format( rul_id)]    =  s_req_fld     # sting with fields required for validation. will be used in a  query later
       globals()['val_l_var_fld_{0}'.format( rul_id)]    =  l_req_fld     # list with fields required for validation .e.g.  ['CTC_NM', 'BIRTH_DT', 'RESIDE_AD_LINE1', ....  'RESIDE_CTRY_CD']

l_req_fld = list(set(re.sub(' +', ' ', _s_req_fld).split()))  # convert a string to a list; convert the list to a set to dedup; convert set to a list
s_req_fld = ', '.join(l_req_fld)  # return a string like 'ID_DOC_ID, ID_DOC_EXPR_DT, ...'

end_time = time.time()  # need to import time
elapse_time (start_time, end_time, dsc)
########################################################################################################################################################
#  Main process starts here
########################################################################################################################################################

start_time = time.time()
dsc = 'Validate business data:'
se10 = '9999999999'  # for testing, set up the se10 to different number for testing

wrk_var     = "toc_se10, se10, abbr_nm, se_lgl_ent_type_cd, ent_grp, bus_rol_grp_abb_1, first_rspbl_cd as rspbl_cd_1,  bus_rol_grp_abb_2, scnd_rspbl_cd as rspbl_cd_2," + \
             "bus_rol_grp_abb_3, third_rspbl_cd as rspbl_cd_3, bus_rol_grp_abb_4, fourth_rspbl_cd as rspbl_cd_4,  bus_rol_grp_abb_5, fifth_rspbl_cd as rspbl_cd_5, bus_ctc_id, " + s_req_fld

wrk_tbl = sch_nm + ".cor_wrk_bus_ctc" + yymm

q_drop = 'drop table if exists {0}'.format(wrk_tbl)
sqlContext.sql(q_drop)

q_cr_wrk_tbl = 'create table {0} as select {1} from {2}  where abbr_nm in {3} and  se10 <= "{4}"'.format(wrk_tbl, wrk_var, _bus_ctc_tbl, s_ctry_nm_in, se10)
sqlContext.sql(q_cr_wrk_tbl)

idx_nm = "wrk_ctc_idx_cmp" + yymm
f_cr_idx(wrk_tbl, idx_nm, "(abbr_nm, se10)", "cmp")

_bus_ctc_tbl = wrk_tbl
q_bus_ctc = 'select * from  {0} sort by abbr_nm, toc_se10, se10'.format(_bus_ctc_tbl)
#q_bus_ctc = 'select * from  {0}  where abbr_nm in {1} and  se10 <= "{2}" sort by abbr_nm, toc_se10, se10'.format(_bus_ctc_tbl, s_ctry_nm_in, se10)  #Testing
sdf_all =  sqlContext.sql(q_bus_ctc).cache()
print (  sdf_all.count(),  " rows to process ")
_l_mis_fld_ary = []
_max_rol_typ_no = 5 # for eact contract ID, the max role type is 5 with index from 1 to 5
_max_rol_typ_no = _max_rol_typ_no + 1
#_l_ctry_nm = ['HUNGARY']   #Testing
 
sel_item = ["toc_se10", "se10","abbr_nm", "se_lgl_ent_type_cd", "ent_grp", "bus_rol_grp_abb_1", "rspbl_cd_1",  "bus_rol_grp_abb_2","rspbl_cd_2",  "bus_rol_grp_abb_3", "rspbl_cd_3",  
            "bus_rol_grp_abb_4", "rspbl_cd_4",   "bus_rol_grp_abb_5", "rspbl_cd_5", "bus_ctc_id"]

for _cty in _l_ctry_nm:
    _l_se_cty  = sdf_all.where( sdf_all.abbr_nm  == _cty ).select(sel_item).toPandas()  # all rows within a country
    
    print ("Records ", len(_l_se_cty),  "for country ", _cty )
    dsc_cty =  str(len(_l_se_cty )) + " records  had been validated for " + _cty + "."
    ratio = 0.2
    cnt = 0.0
    tot_cnt = len(_l_se_cty)
    cty_start_time = time.time()
    cty_time_start = cty_start_time  
    for ix in range(tot_cnt):
        cnt = cnt + 1
        per_cnt = cnt / tot_cnt  
        if per_cnt  >= ratio:  
           cty_end_time = time.time()
           elapsed =  cty_end_time -  cty_start_time
           print ("Complete %5.2f." %( per_cnt), "It took %3f second." %(elapsed))
           ratio = ratio + 0.2
           cty_start_time = cty_end_time        

           
        _se10 = _l_se_cty.loc[ix, "se10"]
        bus_ctc_id      = _l_se_cty.loc[ix, "bus_ctc_id"]
        _toc_se10       = _l_se_cty.loc[ix, "toc_se10"]
        _rol_cd_1       = _l_se_cty.loc[ix, "rspbl_cd_1"]
        _rol_grp_abb_1  = _l_se_cty.loc[ix, "bus_rol_grp_abb_1"]
        _rol_cd_2       = _l_se_cty.loc[ix, "rspbl_cd_2"]
        _rol_grp_abb_2  = _l_se_cty.loc[ix, "bus_rol_grp_abb_2"]
        _rol_cd_3       = _l_se_cty.loc[ix, "rspbl_cd_3"]        
        _rol_grp_abb_3  = _l_se_cty.loc[ix, "bus_rol_grp_abb_3"]
        _rol_cd_4       = _l_se_cty.loc[ix, "rspbl_cd_4"]
        _rol_grp_abb_4  = _l_se_cty.loc[ix, "bus_rol_grp_abb_4"]
        _rol_cd_5       = _l_se_cty.loc[ix, "rspbl_cd_5"]
        _rol_grp_abb_5  = _l_se_cty.loc[ix, "bus_rol_grp_abb_5"]  
       
        for j in range(1, _max_rol_typ_no):
              _rol_typ_cd         = globals()['_rol_cd_{0}'.format(j) ]    
              _rol_typ_grp        = globals()['_rol_grp_abb_{0}'.format(j) ]
              _rol_grp_abb        = globals()['_rol_grp_abb_{0}'.format(j) ]
              # the _rol_typ_cd might defined as '   ', or Null
              if  ( _rol_typ_cd  != None













Type and _rol_typ_cd  != '   ' and   _rol_grp_abb  != None):           
                  cty_nm      =  _cty.lower()
                  _ent_grp           = _l_se_cty.loc[ix, "ent_grp"]
                  _se_lgl_ent_type_cd= _l_se_cty.loc[ix, "se_lgl_ent_type_cd"]
                  _ent_cd_desc  = d_ent_dsc[int(_se_lgl_ent_type_cd)] + "- " + _se_lgl_ent_type_cd  
                  _rol_typ_cd_dsc = d_bus_rol_dsc[int(_rol_typ_cd) ]  
                  rul_id      =  cty_nm + str(_ent_grp) +  _rol_typ_grp.lower()
                  req_cd      =  globals()['val_req_cd_{0}'.format( rul_id) ]  
                  if (req_cd != 'o' ):  
                      s_val_fld   =  globals()['val_s_var_fld_{0}'.format( rul_id)]
                      l_val_fld   =  ['se10' ] + globals()['val_l_var_fld_{0}'.format( rul_id)]  
                      no_val_fld  =  globals()['val_no_var_fld_{0}'.format( rul_id) ]
               
                      l_bus_data =   sdf_all.where( sdf_all.se10 == _se10).where(sdf_all.toc_se10 == _toc_se10).where(sdf_all.bus_ctc_id   == bus_ctc_id ).select(l_val_fld).toPandas()
                                                              
                      #fld_len  = no_val_fld
                      fld_data = l_bus_data.iloc[0, 1:no_val_fld + 1]   # return type as tuple; get all the values for the fields  related to field ID from SQL result get all the values required ftype (fld_datal_data = [ x if x is None else x.strip() for x in list(fld_data) ]
                      l_data = [ x if x is None else x.strip() for x in list(fld_data) ]
                      data_isnull    = list(map(f_isnull,l_data))
                      no_fld_eq_null = sum(map(f_isnull,l_data))
                      #print(_se10, "fld_data", fld_data, "no_fld_eq_null", no_fld_eq_null, "no_fld_eq_null", no_fld_eq_null , '\n',  "no_val_fld", no_val_fld, "req_cd", req_cd)
                                      
                      if (req_cd == 'm' and no_fld_eq_null > 0 ) or (req_cd == 'c' and no_fld_eq_null > 0 and no_fld_eq_null  !=  no_val_fld) :
                                                
                         mis_sumy= ""
                         mis_sumy_det = "Missing Field: "
                         iz = 0
                         for iy in range(no_val_fld ):
                              
                             if data_isnull[iy]:
                                iz  = iz + 1
                                fld_name = s_val_fld.strip().split(',')[iy]                  
                                mis_sumy = mis_sumy + fld_name
                                if iz == 1:
                                   mis_sumy_det = mis_sumy_det + d_fld_nam[fld_name.strip()]
                                else:    
                                   mis_sumy_det = mis_sumy_det + ", " + d_fld_nam[fld_name.strip()]
                                             
                         _l_mis_fld_ary.append((_toc_se10, _se10, _se_lgl_ent_type_cd, _ent_grp, _ent_cd_desc, _rol_typ_cd, _rol_typ_grp, _rol_typ_cd_dsc, mis_sumy, mis_sumy_det ))
      
    end_time = time.time()
    elapse_time (cty_time_start, end_time, dsc_cty)                        

end_time = time.time()  # need to import time
elapse_time (start_time, end_time, dsc)

#vav - load data
df_defect = sqlContext.createDataFrame(sc.parallelize(_l_mis_fld_ary), schema)
sqlContext.registerDataFrameAsTable(df_defect, "cor_defect")
sqlContext.registerDataFrameAsTable(df_defect, "cor_defect")
defect_tbl= sch_nm + ".cor_wrk_defect_bus_data_" + yymm

q_drop = "drop table if exists {0}".format(defect_tbl)
sqlContext.sql(q_drop)
q_cr_tbl = "create table {0} as select * from cor_defect".format(defect_tbl)
sqlContext.sql(q_cr_tbl)

f_get_tbl_cnt(defect_tbl, "")
job_end_time = time.time()
job_dsc = "Validate business data for all countries."
elapse_time (job_start_time, job_end_time, job_dsc)
print ("End of " + pgm)
cor_rpt_mrg_defect_tbl.py
pgm = "cor_rpt_mrg_defect_tbl.py"
path_code = "/axp/buanalytics/csramp/dev/cor/code/"
pgm = "cor_bld_defect_rpt.py"
print ("Start of " + pgm)
################################################################################################################################################
#ini - Initialization
################################################################################################################################################

# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))
job_start_time = time.time()
u_get_mis_sumy_for_mis_role= udf(f_get_mis_sumy_for_mis_role)

################################################################################################################################################
#  Main process starts here
################################################################################################################################################

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Combine business role defect table and business data defect table into a defect table
#-----------------------------------------------------------------------------------------------------------------------------------------------

bus_ctc_tbl = sch_nm + ".cor_bus_ctc_" + yymm
wrk_var     = "dfct.toc_se10 ,dfct.se10, ctc.se_dba_nm,  dfct.ent_cd_dsc, dfct.rol_typ_cd, dfct.rol_typ_cd_dsc, dfct.mis_sumy_det, dfct.rol_typ_grp"
dfct_tbl    = sch_nm + ".cor_defect_" + yymm
dfct_role_tbl  =  sch_nm + ".cor_wrk_defect_bus_role_" + yymm
q_bus_role = "select * from {0}".format(dfct_role_tbl)
sdf_bus_role = sqlContext.sql (q_bus_role)
sdf_bus_role_new = sdf_bus_role.drop("mis_sumy").withColumn('mis_sumy',u_get_mis_sumy_for_mis_role("mis_sumy_det") ).select("toc_se10", "se10", "se_lgl_ent_type_cd", "ent_grp", "ent_cd_dsc",  "rol_typ_cd", "rol_typ_grp", "rol_typ_cd_dsc", "mis_sumy", "mis_sumy_det")

sdf_bus_role_new.registerTempTable('tmp_bus_rol_tbl')
dfct_role_tbl = "tmp_bus_rol_tbl"
dfct_data_tbl  =  sch_nm + ".cor_wrk_defect_bus_data_" + yymm
q_drop = 'drop table {0}'.format(dfct_tbl)
sqlContext.sql(q_drop)

q_cr_dfct_tbl  = """ create table {0}  as  select * from
                    (select  * from {1}
                     union all
                     select  *  from {2}
                     ) tmp
                """.format(dfct_tbl,  dfct_role_tbl, dfct_data_tbl )
sqlContext.sql(q_cr_dfct_tbl)
f_get_tbl_cnt(dfct_tbl, "")

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Combine a table to get the merchant name for reporting pupose
#-----------------------------------------------------------------------------------------------------------------------------------------------

dfct_mer_nm_tbl = sch_nm + ".cor_wrk_dfct_mer_nm_" + yymm
q_drop = 'drop table {0}'.format (dfct_mer_nm_tbl)
sqlContext.sql(q_drop)

q_cr_dfct_mer_nm = """create table {0} as
                    select {1}
                    from  {2} dfct
                    left  outer join {3} ctc on dfct.se10 = ctc.se10
                    group by {1}""".format(dfct_mer_nm_tbl, wrk_var, dfct_tbl, bus_ctc_tbl)
sqlContext.sql(q_cr_dfct_mer_nm )

f_get_tbl_cnt(dfct_mer_nm_tbl, '')
job_end_time = time.time()
elapse_time (job_start_time, job_end_time, pgm)
print ("end of " + pgm )      
                    
cor_rpt_defect_rpt.py
pgm = "cor_rpt_defect_rpt.py"
print ("Start of " + pgm)
################################################################################################################################################
#ini - Initialization
################################################################################################################################################

# Invoke code to get parm to determine the location of the folder, schema name and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
print(" bef init" ,  'path_code', path_code, '\n', 'path_data', path_data)
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))
job_start_time = time.time()
print(" aft init" ,  'path_code', path_code, '\n', 'path_data', path_data)

################################################################################################################################################
#  Main process starts here
################################################################################################################################################


#-----------------------------------------------------------------------------------------------------------------------------------------------
#Create a dataframe from the defect table required for excel file
#-----------------------------------------------------------------------------------------------------------------------------------------------
dfct_tbl = sch_nm +  ".cor_wrk_dfct_mer_nm_" + yymm

q_sel = "select * from {0}".format(dfct_tbl)
sdf_dfct= sqlContext.sql(q_sel)
_l_mis_ary = sdf_dfct.collect()

labels =  ["Pay Manager SE10", "Merchant SE10",  "Merchant Name",  "Legal Entity", "Role Code",  \
           "Role Code Description",  "Quality  Summary (Missing fields)", "_rol_typ_grp" ]

df_from_list=pd.DataFrame.from_records(_l_mis_ary, columns = labels)              # create a pd  dataframe
df_from_list.sort_values(['Merchant SE10', '_rol_typ_grp' ], inplace = True)     
#df_from_list.sort(['Merchant SE10', '_rol_typ_grp' ], inplace = True)            # "AttributeError: 'DataFrame' object has no attribute 'sort'; use 'sort_values' instead
df_from_list.drop(['_rol_typ_grp'], axis=1, inplace=True)                         # drop no neeed columns; cause drop() got an unexpected keyword argument 'inplace'

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Build up a look up dictionary to get ctry_abbr_nm from toc_se10
# - There is no corresponding table for cor_lup_partner_grp.csv which is used by reporting only
#-----------------------------------------------------------------------------------------------------------------------------------------------
print(" bef f_lup" ,  'path_code', path_code, '\n', 'path_data', path_data)

f_lup  = path_data + "cor_lup_partner_grp.csv"
df_lup = pd.read_csv(f_lup)
df_lup.sort_values(['partner' ],  inplace = True)

l_val  = df_lup['ctry_abbr_nm'].tolist()
l_key  = df_lup['toc_se10'].tolist()
d_toc_se10_ctry  = dict (zip(l_key, l_val))

#-----------------------------------------------------------------------------------------------------------------------------------------------
#create report
#-----------------------------------------------------------------------------------------------------------------------------------------------


nxt_yymm = f_dte_for_yymm(yymm, 'n')
mmddccyy = f_dte_for_yymm(nxt_yymm, 'f2')

for _partner  in df_lup['partner'].unique():
    df_lup_new = df_lup[df_lup['partner'] == _partner]
    # print("df_lup_new", df_lup_new, "_partner",  _partner)
    _w_file_wrt_ind = 0
    for _toc_se in  df_lup_new["toc_se10"].unique():       
        df_toc_se = df_from_list[df_from_list["Pay Manager SE10"] == str(_toc_se)]
        _w_count = df_from_list[df_from_list["Pay Manager SE10"] == str(_toc_se)]["Pay Manager SE10"].count()
       
        if  _w_count > 0:
            if _w_file_wrt_ind == 0 :
               excel_file = path_data +  str(_partner) + "_" + str(mmddccyy) + ".xlsx"
               print ("excel_file ", excel_file )
               writer = pd.ExcelWriter(excel_file)
               _w_file_wrt_ind = 1
            ctry = d_toc_se10_ctry[_toc_se]                  
            sheet_name =  str(_toc_se) + "_" + ctry            
            df_toc_se.to_excel(writer,sheet_name, index=False)     
               
    if   _w_file_wrt_ind == 1:
         writer.save()


job_end_time = time.time()

elapse_time (job_start_time, job_end_time, pgm)
print ("end of " + pgm )       

cor_prg_bld_trk.py

pgm = "cor_prg_bld_trk.py"
print ("Start of " + pgm)
################################################################################################################################################
#ini - Initialization
################################################################################################################################################

# Invoke code to get parm to determine the location of the folder and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))
job_start_time = time.time()

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Build dictionary
# - Invoke function 'f_cr_dic_from_csv' to build dictionary d_fld_nam  to map the field name from CS to bus field name
# - Invoke 'f_cr_dic_from_tbl' to get bus role description
#-----------------------------------------------------------------------------------------------------------------------------------------------


csv_file  = path_data + 'cor_lup_corns_fld.csv'
#global d_bus_rol_grp_dsc, d_bus_rol_grp_dsc
d_fld_nam =  f_cr_dic_from_csv(csv_file, 'bus_fld_nam', 'corns_fld_nam')

bus_rul_tbl = sch_nm + ".cor_lup_bus_rule"
d_bus_rol_grp_dsc =  f_cr_dic_from_tbl(bus_rul_tbl, "rol_typ", "rol_typ_dsc")

################################################################################################################################################
#  Main process starts here
################################################################################################################################################


#-----------------------------------------------------------------------------------------------------------------------------------------------
#Create a dataframe from the defect table required for excel file
#-----------------------------------------------------------------------------------------------------------------------------------------------

cur_yymm = yymm
prv_yymm = f_dte_for_yymm(yymm, 'p')
print ("process " + yymm + " data." + " Previous month is " + prv_yymm )


cur_dfct_tbl = sch_nm + ".cor_defect_" + str(cur_yymm)
prv_dfct_tbl = sch_nm + ".cor_defect_" + str(prv_yymm)
prv_prg_trk_tbl =  sch_nm + ".cor_dfct_prg_trk_"  + str (prv_yymm)


q_sdf_dfct_key_cur =  "select toc_se10, se10, rol_typ_cd  from {0} group by toc_se10, se10, rol_typ_cd".format(cur_dfct_tbl)
sdf_dfct_key_cur  =  sqlContext.sql(q_sdf_dfct_key_cur)   #sdf

# will use toc_se10, se10, rol_typ_cd as key to check the completion and new defect
q_sdf_dfct_key_prv =  "select toc_se10, se10, rol_typ_cd  from {0} group by toc_se10, se10, rol_typ_cd".format(prv_dfct_tbl)
sdf_dfct_key_prv  =  sqlContext.sql(q_sdf_dfct_key_prv)   #sdf

sdf_cmp_key_dfct  = sdf_dfct_key_prv.subtract(sdf_dfct_key_cur)  #
sdf_cmp_key_dfct.registerTempTable('tmp_cmp_key_dfct_tbl')  #register temp table
print("Count for toc_se10,  se10, rol_typ_cd  in previous defect report only = ",   f_get_tbl_cnt('tmp_cmp_key_dfct_tbl', ''))
      
sdf_new_key_dfct  = sdf_dfct_key_cur.subtract(sdf_dfct_key_prv)  #
sdf_new_key_dfct.registerTempTable('tmp_new_key_dfct_tbl')  #register temp table
print("Count for toc_se10,  se10, rol_typ_cd  in current defect report only = ",   f_get_tbl_cnt('tmp_new_key_dfct_tbl', ''))
     

sel_fld_cur = "{0}.toc_se10, {0}.se10, {0}.se_lgl_ent_type_cd, {1} as dfct_srt_dte, {0}.ent_cd_dsc, {0}.rol_typ_cd   ,{0}.rol_typ_grp  ,{0}.rol_typ_cd_dsc  ".format('cur', prv_yymm)
sel_fld_prv = "{0}.toc_se10, {0}.se10, {0}.se_lgl_ent_type_cd, {1} as dfct_srt_dte, {0}.ent_cd_dsc, {0}.rol_typ_cd   ,{0}.rol_typ_grp  ,{0}.rol_typ_cd_dsc  ".format('prv', prv_yymm)

#Create defect new table
print("Create defect new table")
dfct_new_tbl =  sch_nm + ".cor_dfct_new_"  + str (cur_yymm)
q_dfct_new = "select cur.* from {0} cur, {1} new  where  cur.toc_se10   = new.toc_se10 and cur.se10  = new.se10 and cur.rol_typ_cd = new.rol_typ_cd".format(cur_dfct_tbl, "tmp_new_key_dfct_tbl")
sqlContext.sql ( q_dfct_new ).registerTempTable('tmp_dfct_new_tbl')
f_cr_tbl_selas (dfct_new_tbl, 'tmp_dfct_new_tbl', '' )


#  get se_sta_cd_ds for merchant had been complete
print("process defect(s) had been completed or canceled")
mernm_tbl  =  sch_nm + ".cor_wrk_mernm_stscd"

q_dfct_cmp = """
select {0}, ' ' as mis_sumy,
case  
  when mer.se_sta_cd_ds like ('%CANCEL%')  then 'CANCEL'
 else 'COMPLETE'
end as mis_sumy_det
from {1} prv,
     {2} mer,
     tmp_cmp_key_dfct_tbl  cmp
where prv.se10 = mer.se10
 and prv.se10 = cmp.se10
 and prv.rol_typ_cd = cmp.rol_typ_cd""".format(sel_fld_prv, prv_dfct_tbl, mernm_tbl )  
sdf_dfct_cmp =  sqlContext.sql(q_dfct_cmp) # sdf for defect completion

#
print("process common defect(s) + new defect for commom merchant ")
q_sdf_com_key = """select cur.toc_se10, cur.se10, cur.rol_typ_cd  from {0} cur, {1} prv
                   where  cur.toc_se10   = prv.toc_se10 and cur.se10  = prv.se10 and cur.rol_typ_cd = prv.rol_typ_cd
                   group by  cur.toc_se10,  cur.se10, cur.rol_typ_cd
                """.format(cur_dfct_tbl, prv_dfct_tbl)

sqlContext.sql (q_sdf_com_key).registerTempTable('tmp_com_key_tbl')
print("Count for toc_se10,  se10, rol_typ_cd  in both current and previous defect reports = ",   f_get_tbl_cnt('tmp_new_key_dfct_tbl', '' ))     
q_sdf_com_dfct = """select {0}, cur.mis_sumy, cur.mis_sumy_det  from {1} cur, {2} com
                    where cur.toc_se10   = com.toc_se10 and cur.se10  = com.se10 and cur.rol_typ_cd = com.rol_typ_cd
                 """.format(sel_fld_cur, cur_dfct_tbl, "tmp_com_key_tbl")
sdf_com_dfct = sqlContext.sql(q_sdf_com_dfct)

f_union_all(sdf_dfct_cmp, sdf_com_dfct).registerTempTable('tmp_prg_trk_tbl')


if f_chk_tbl_exist(prv_prg_trk_tbl) and f_get_tbl_cnt(prv_prg_trk_tbl, '') > 0  :
   q_sdf_upd_dfct_srt_dte   = """ select  cur.toc_se10, cur.se10, cur.se_lgl_ent_type_cd,
                                          coalesce( prv.dfct_srt_dte,  cur.dfct_srt_dte) as  dfct_srt_dte,
                                          cur.ent_cd_dsc, cur.rol_typ_cd, cur.rol_typ_grp, cur.rol_typ_cd_dsc, cur.mis_sumy, cur.mis_sumy_det
                                  from tmp_prg_trk_tbl cur
                                  left outer join {0} prv
                                  on cur.toc_se10   = prv.toc_se10 and cur.se10  = prv.se10 and cur.rol_typ_cd = prv.rol_typ_cd
                             """.format(prv_prg_trk_tbl )
   sqlContext.sql(q_sdf_upd_dfct_srt_dte).registerTempTable('tmp_prg_trk_tbl')

prg_trk_tbl =  sch_nm + ".cor_dfct_prg_trk_"  + str (cur_yymm)
f_cr_tbl_selas (prg_trk_tbl, 'tmp_prg_trk_tbl', '' )
   
                                                                                                       
job_end_time = time.time()
elapse_time (job_start_time, job_end_time, pgm)                                                                                                       

print ("End of " + pgm)

cor_prg_trk_rpt.py
pgm = "cor_prg_trk_rpt.py"
print ("Start of " + pgm)

# Invoke code to get parm to determine the location of the folder and yymm
path_parm_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_parm_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))

#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

################################################################################################################################################
#  Main process starts here
################################################################################################################################################

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Build up a look up dictionary to get ctry_abbr_nm from toc_se10
# - There is no corresponding table for cor_lup_partner_grp.csv which is used by reporting only
#-----------------------------------------------------------------------------------------------------------------------------------------------

csv_file  = path_data + "cor_lup_partner_grp.csv"
d_toc_se10_ctry =  f_cr_dic_from_csv(csv_file, 'ctry_abbr_nm', 'toc_se10')
df_lup = pd.read_csv(csv_file)
df_lup.sort_values(['partner' ],  inplace = True)


mer_tbl  =  sch_nm + ".cor_wrk_mernm_stscd"
prg_trk_tbl  =  sch_nm + ".cor_dfct_prg_trk_"  + str (yymm)
dfct_new_tbl =  sch_nm + ".cor_dfct_new_"      + str (yymm)  

sqlContext.udf.register("udf_get_mth_diff", f_get_mth_diff)
#-----------------------------------------------------------------------------------------------------------------------------------------------
# get the merchant name for progress tracking
#-----------------------------------------------------------------------------------------------------------------------------------------------

q_sdf_trk_mer = """select trk.toc_se10, trk.se10, mer.se_dba_nm, trk.ent_cd_dsc, trk.rol_typ_cd, trk.rol_typ_cd_dsc, trk.dfct_srt_dte, trk.mis_sumy_det, trk.rol_typ_grp, udf_get_mth_diff(dfct_srt_dte, '{2}' ) as mth_diff
                     from {0} trk
                     left outer join  {1} mer
                     on    trk.se10 = mer.se10                     
                     sort by trk.se10,trk.rol_typ_grp
                 """.format(prg_trk_tbl, mer_tbl, yymm)
print("q_sdf_trk_mer", q_sdf_trk_mer)
sdf_trk_mer = sqlContext.sql(q_sdf_trk_mer)


_date = f_dte_for_yymm(yymm, 'd')
ost_mmm_ccyy = "Outstanding as of " + calendar.month_abbr[_date.month] + '-' + str(_date.year)


_l_mis_ary = sdf_trk_mer.collect()
labels =  ["Pay Manager SE10", "Merchant SE10",  "Merchant Name",  "Legal Entity", "Role Code",  \
           "Role Code Description", "Date of first Occurrence of defect",  ost_mmm_ccyy,  "rol_typ_grp", "mth_diff" ]

df_trk=pd.DataFrame.from_records(_l_mis_ary, columns = labels)              # create a pd  dataframe
df_trk.sort_values(['Merchant SE10', 'rol_typ_grp' ], inplace = True)     
 
df_trk.drop(['rol_typ_grp'], axis=1, inplace=True)

df_prg_trk   = df_trk[ df_trk['mth_diff'] < '4' ]
df_90p_trk = df_trk[df_trk['mth_diff']    > '3' ]

if not pd.DataFrame(df_prg_trk).empty:
 df_prg_trk.drop(['mth_diff'], axis=1, inplace=True)
                  
if not pd.DataFrame(df_90p_trk).empty:
 df_90p_trk.drop(['mth_diff'], axis=1, inplace=True) 

#-----------------------------------------------------------------------------------------------------------------------------------------------
# get the merchant name for new defect
# Notes:
#  - writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')  # Convert the dataframe to an XlsxWriter Excel object
#    - engine='xlsxwriter' is required to define workbook and woeksheett to adjust the width of columns
#-----------------------------------------------------------------------------------------------------------------------------------------------

q_sdf_dfct_new_mer = """select dfct.toc_se10, dfct.se10, mer.se_dba_nm, dfct.ent_cd_dsc, dfct.rol_typ_cd, dfct.rol_typ_cd_dsc, dfct.mis_sumy_det, dfct.rol_typ_grp
                     from {0} dfct       
                     left outer join  {1} mer
                     on    dfct.se10 = mer.se10                     
                     sort by dfct.se10, dfct.rol_typ_grp
                 """.format(dfct_new_tbl, mer_tbl)
print("q_sdf_dfct_new_mer", q_sdf_dfct_new_mer)
_l_mis_ary = sqlContext.sql(q_sdf_dfct_new_mer).collect()


labels =  ["Pay Manager SE10", "Merchant SE10",  "Merchant Name",  "Legal Entity", "Role Code",  \
           "Role Code Description",  "Quality  Summary (Missing fields)", "rol_typ_grp" ]

df_new_dfct = pd.DataFrame.from_records(_l_mis_ary, columns = labels)              # create a pd  dataframe
df_new_dfct.sort_values(['Merchant SE10', 'rol_typ_grp' ], inplace = True)    

df_new_dfct.drop(['rol_typ_grp'], axis=1, inplace=True)  

#-----------------------------------------------------------------------------------------------------------------------------------------------
#create report
#-----------------------------------------------------------------------------------------------------------------------------------------------

nxt_yymm = f_dte_for_yymm(yymm, 'n')
mmddccyy = f_dte_for_yymm(nxt_yymm, 'f2')
s_partner =   df_lup[df_lup ["status"] == 'A']["partner"].unique()
for _partner  in s_partner:
    df_lup_new = df_lup[df_lup['partner'] == _partner]
    print("df_lup_new", df_lup_new, "_partner",  _partner)
    _w_file_wrt_ind = 0
    for _toc_se in  df_lup_new["toc_se10"].unique():
        df_toc_se_new_dfct = df_new_dfct[df_new_dfct["Pay Manager SE10"] == str(_toc_se)]
        df_toc_se_prg_trk  = df_prg_trk[df_prg_trk["Pay Manager SE10"]   == str(_toc_se)]
        df_toc_se_90p_trk  = df_90p_trk[df_90p_trk["Pay Manager SE10"]   == str(_toc_se)]
        
        _w_count_new_dfct  = df_new_dfct[df_new_dfct["Pay Manager SE10"] == str(_toc_se)]["Pay Manager SE10"].count()
        _w_count_prg_trk   = df_prg_trk[df_prg_trk["Pay Manager SE10"]   == str(_toc_se)]["Pay Manager SE10"].count()
        _w_count_90p_trk   = df_90p_trk[df_90p_trk["Pay Manager SE10"]   == str(_toc_se)]["Pay Manager SE10"].count()        
        print("_w_count_new_dfct", _w_count_new_dfct, "_w_count_prg_trk", _w_count_prg_trk)
        if  _w_count_new_dfct > 0 or  _w_count_prg_trk > 0  or _w_count_90p_trk > 0 :
            if _w_file_wrt_ind == 0 :
               excel_file = path_data +  str(_partner) + "_progress_" + str(mmddccyy) + ".xlsx"
               print ("excel_file ", excel_file )   
               writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')  # Convert the dataframe to an XlsxWriter Excel object
               _w_file_wrt_ind = 1
            ctry = d_toc_se10_ctry[_toc_se]
            
            if _w_count_new_dfct > 0:
               sheet_name =  str(_toc_se) + "_" + ctry + '_new_defect'
               df_toc_se_new_dfct.to_excel(writer,sheet_name, index=False)
               f_adj_col_sheet(df_toc_se_new_dfct, writer, sheet_name, 'True')
               
            if _w_count_prg_trk > 0:
               sheet_name =  str(_toc_se) + "_" + ctry + '_progress'
               df_toc_se_prg_trk.to_excel(writer,sheet_name, index=False)
               f_adj_col_sheet(df_toc_se_prg_trk, writer, sheet_name, 'True')
               
            if _w_count_90p_trk > 0:
               sheet_name =  str(_toc_se) + "_" + ctry + '_90plus'
               df_toc_se_90p_trk.to_excel(writer,sheet_name, index=False)
               f_adj_col_sheet(df_toc_se_90p_trk, writer, sheet_name, 'True')
               

    if   _w_file_wrt_ind == 1:
         writer.save()
    else:
         excel_file = path_data +  str(_partner) + "_progress_" + str(mmddccyy) + ".xlsx"
         print ("excel_file ", excel_file )
         writer = pd.ExcelWriter(excel_file)
         writer.save()


pgm = "cor_chg_trk_rpt.py"
print ("End of " + pgm)


cor_chg_bld_trk.py
pgm = "cor_chg_bld_trk.py"
print ("Start of " + pgm)
################################################################################################################################################
#ini - Initialization
################################################################################################################################################

#load code for to get parm
path_code = "/axp/buanalytics/csramp/dev/cor/code/"
#path_data = "/axp/buanalytics/csramp/dev/cor/data_prod/"
f_parm = path_code + "cor_com_code_parm.py"
exec(compile(open(f_parm, "rb").read(), f_parm, 'exec' ))


#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))
job_start_time = time.time()

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Register udf
# - the functions are defined in  cor_com_func.py
#-----------------------------------------------------------------------------------------------------------------------------------------------

sqlContext.udf.register("udf_com_fld", f_com_fld)
sqlContext.udf.register("udf_dif_fld", f_dif_fld)
sqlContext.udf.register("udf_bld_mis_sumy_det", f_bld_mis_sumy_det)
u_bld_mis_sumy_det= udf(f_bld_mis_sumy_det)


#-----------------------------------------------------------------------------------------------------------------------------------------------
#Build dictionary
# - Invoke function 'f_cr_dic_from_csv' to build dictionary d_fld_nam  to map the field name from CS to bus field name
# - Invoke 'f_cr_dic_from_tbl' to get bus role description
#-----------------------------------------------------------------------------------------------------------------------------------------------

path_data = '/axp/buanalytics/csramp/dev/cor/data/'
csv_file  = path_data + 'cor_lup_corns_fld.csv'
#global d_bus_rol_grp_dsc, d_bus_rol_grp_dsc
d_fld_nam =  f_cr_dic_from_csv(csv_file, 'bus_fld_nam', 'corns_fld_nam')

bus_rul_tbl = "pcr_cor.cor_lup_bus_rule"
d_bus_rol_grp_dsc =  f_cr_dic_from_tbl(bus_rul_tbl, "rol_typ", "rol_typ_dsc")

################################################################################################################################################
#  Main process starts here
################################################################################################################################################

#-----------------------------------------------------------------------------------------------------------------------------------------------
#Create a dataframe from the defect table required for excel file
#-----------------------------------------------------------------------------------------------------------------------------------------------
yymm = (pd.Period(datetime.today(), 'M') - 1).strftime('%y%m')

yymm = 1811
sch_nm = "yuedb"
cur_yymm= yymm
prv_yymm= 1810   # will calculate from cur_yymm later

cur_dfct_tbl = sch_nm + ".cor_defect_" + str(cur_yymm)
prv_dfct_tbl = sch_nm + ".cor_defect_" + str(prv_yymm)

#chg1
cur_sumy = 1811
prv_yymm = 1810

cur_dfct_tbl = "yuedb.csv_wrk_defect_1811"
prv_dfct_tbl = "yuedb.csv_wrk_defect_1810"

q_sdf_dfct_key_cur =  "select toc_se10, se10, rol_typ_cd  from {0}".format(cur_dfct_tbl)
sdf_dfct_key_cur  =  sqlContext.sql(q_sdf_dfct_key_cur)   #sdf

# will use toc_se10, se10, rol_typ_cd as key to check the completion and new defect
q_sdf_dfct_key_prv =  "select toc_se10, se10, rol_typ_cd  from {0}".format(prv_dfct_tbl)
sdf_dfct_key_prv  =  sqlContext.sql(q_sdf_dfct_key_prv)   #sdf

sdf_new_key_dfct  = sdf_dfct_key_cur.subtract(sdf_dfct_key_prv)  # 6
sdf_cmp_key_dfct  = sdf_dfct_key_prv.subtract(sdf_dfct_key_cur)  #1
sdf_cmp_key_dfct.registerTempTable('tmp_cmp_key_dfct_tbl')  #register temp table
sdf_new_key_dfct.registerTempTable('tmp_new_key_dfct_tbl')  #register temp table

from_where_cond = "from {0}  cur, {1}  prv where  cur.toc_se10   = prv.toc_se10 and cur.se10  = prv.se10 and cur.rol_typ_cd = prv.rol_typ_cd".format(cur_dfct_tbl, prv_dfct_tbl  )
sel_fld_cur = "{0}.toc_se10, {0}.se10, {0}.se_lgl_ent_type_cd, {1} as dfct_srt_dte, {0}.ent_cd_dsc, {0}.rol_typ_cd   ,{0}.rol_typ_grp  ,{0}.rol_typ_cd_dsc  ".format('cur', prv_yymm)
sel_fld_prv = "{0}.toc_se10, {0}.se10, {0}.se_lgl_ent_type_cd, {1} as dfct_srt_dte, {0}.ent_cd_dsc, {0}.rol_typ_cd   ,{0}.rol_typ_grp  ,{0}.rol_typ_cd_dsc  ".format('prv', prv_yymm)

#Create defect new table
print("Create defect new table")
dfct_new_tbl =  sch_nm + ".cor_dfct_new_"  + str (cur_yymm)
q_dfct_new = "select cur.* from {0} cur, {1} new  where  cur.toc_se10   = new.toc_se10 and cur.se10  = new.se10 and cur.rol_typ_cd = new.rol_typ_cd".format(cur_dfct_tbl, "tmp_new_key_dfct_tbl")
sqlContext.sql ( q_dfct_new ).registerTempTable('tmp_dfct_new_tbl')
f_cr_tbl_selas (dfct_new_tbl, 'tmp_dfct_new_tbl', '' )


#  get se_sta_cd_ds for merchant had been complete
print("process defect(s) had been completed or canceled")
q_dfct_cmp = """
select {0}, ' ' as mis_sumy,
case  
  when mer.se_sta_cd_ds in ('CANCEL')  then 'CANCEL'
 else 'COMPLETE'
end as mis_sumy_det
from {1} prv,
     cstonedb3.gms_merchant_char mer,
     tmp_cmp_key_dfct_tbl  cmp
where prv.se10 = mer.se10
 and prv.se10 = cmp.se10
 and prv.rol_typ_cd = cmp.rol_typ_cd""".format(sel_fld_prv, prv_dfct_tbl)  
sdf_dfct_cmp =  sqlContext.sql(q_dfct_cmp) # sdf for defect completion

#
print("process common defect(s) + new defect for commom merchant ")
q_sdf_com_key = """select cur.toc_se10, cur.se10, cur.rol_typ_cd  from {0} cur, {1} prv
                   where  cur.toc_se10   = prv.toc_se10 and cur.se10  = prv.se10 and cur.rol_typ_cd = prv.rol_typ_cd
                """.format(cur_dfct_tbl, prv_dfct_tbl)

sqlContext.sql (q_sdf_com_key).registerTempTable('tmp_com_key_tbl')
q_sdf_com_dfct = """select {0}, cur.mis_sumy, cur.mis_sumy_det  from {1} cur, {2} com
                    where cur.toc_se10   = com.toc_se10 and cur.se10  = com.se10 and cur.rol_typ_cd = com.rol_typ_cd
                 """.format(sel_fld_cur, cur_dfct_tbl, "tmp_com_key_tbl")
sdf_com_dfct = sqlContext.sql(q_sdf_com_dfct)

f_union_all(sdf_dfct_cmp, sdf_com_dfct).registerTempTable('tmp_trk_90D_tbl')
                                                                                                      
trk_90d_tbl =  sch_nm + ".cor_dfct_trk_90d_"  + str (cur_yymm)                                                                                                      
f_cr_tbl_selas (trk_90d_tbl, 'tmp_trk_90D_tbl', '' )

                                                                                                       
job_end_time = time.time()
elapse_time (job_start_time, job_end_time, pgm)                                                                                                       

print ("End of " + pgm)

cor_chg_trk_rpt.py
In [ ]:
 
pgm = "cor_chg_trk_rpt.py"
print ("Start of " + pgm)
path_data = "/axp/buanalytics/csramp/dev/cor/data/"
path_code = "/axp/buanalytics/csramp/dev/cor/code/"



#load code for Initialization
f_init = path_code + "cor_com_code_init.py"
exec(compile(open(f_init, "rb").read(), f_init, 'exec' ))

mer_tbl ="cstonedb3.gms_merchant_char"
trk_tbl = "yuedb.cor_dfct_trk_90d_1811"
dfct_new_tbl = "yuedb.cor_dfct_new_1811"
q_mer_nm = "select mer.se10, mer.se_dba_nm from {1} trk, {2}  new, {0} mer  where  mer.se10 = trk.se10 or mer.se10 = new.se10 group by  mer.se10, mer.se_dba_nm".format (mer_tbl, trk_tbl, dfct_new_tbl)


pdf = sqlContext.sql(q_mer_nm).toPandas()
l_key = pdf[se10].tolist()
l_val = pdf[se_dba_nm].tolist()
d_mer_nm = dict (zip(l_key, l_val))
def f_get_mer_nm(se10):
    return d_mer_nm[se10]



u_get_mer_nm = udf(f_get_mer_nm)
sdf_trk = sqlContext.sql( "select * from yuedb.cor_dfct_trk_90d_1811" ).withColumn('mer_nm', u_get_mer_nm("se10") )

l_mis_ary = sdf_trk.select("toc_se10", "se10", "mer_nm", "ent_cd_dsc", "rol_typ_grp", "rol_typ_cd_dsc", "dfct_srt_dte", "mis_sumy_det ", "rol_typ_grp"  ).collect()
labels =  ["Pay Manager SE10", "Merchant SE10",  "Merchant Name",  "Legal Entity", "Role Code",  \
           "Role Code Description", "Date of first Occurrence of defect",  "Outstanding as of NOV-2018", "_rol_typ_grp" ]

df_from_list=pd.DataFrame.from_records(_l_mis_ary, columns = labels)              # create a pd  dataframe
df_from_list.sort_values(['Merchant SE10', '_rol_typ_grp' ], inplace = True)     
#df_from_list.sort(['Merchant SE10', '_rol_typ_grp' ], inplace = True)            # "AttributeError: 'DataFrame' object has no attribute 'sort'; use 'sort_values' instead
df_from_list.drop(['_rol_typ_grp'], axis=1, inplace=True)  

excel_file = path_data +  'tracking_90d ' + "_" + "1811.xlsx"
print ("excel_file ", excel_file )
writer = pd.ExcelWriter(excel_file)
df_from_list.to_excel(writer,tbl, index=False)
writer.save()

pgm = "cor_chg_trk_rpt.py"
print ("End of " + pgm)
 
 
# Example Network SSH script for
# To get netmiko and xlsxwriter:
# open command prompt and enter:
#   - C:\Users\<ADS_ID_HERE>\AppData\Local\Programs\Python\Python36\Scripts
#   - Enter: pip install netmiko --proxy=proxy.aexp.com:8080
#   - Enter: pip install xlsxwriter --proxy=proxy.aexp.com:8080
 
import netmiko
import xlsxwriter
import threading
import time
 
hostlist = ['172.31.8.100', '172.31.0.224']
 
cred = []
file = open(r'C:\Users\mjureck\PycharmProjects\NAOI_OFFICIAL\TRAINING\Python_DataScience\cred.txt', 'r')
for line in file:
    line = line.rstrip('\n')
    line = line.strip()
    cred.append(line)
 
file.close()
 
username = cred[0]
password = cred[1]
ios_ver = 'cisco_ios'
 
outputlist = []
 
def thread(u, p):
    threads = []
    for ipaddress in hostlist:
        t = threading.Thread(target=ssh, args=(ipaddress, username, password,), )
        t.start()
        threads.append(t)
        print('Starting Connection to:', ipaddress)
        time.sleep(1)
    for thread in threads:
        thread.join()
 
def ssh(ipaddress, username, password):
    try:
        net_connect = netmiko.ConnectHandler(ip=ipaddress, username=username, password=password, secret=password,
                                             device_type=ios_ver)
 
        cmd = net_connect.send_command('show run | inc flow')
        output = ipaddress + '###' + cmd
        outputlist.append(output)
        print(cmd)
 
        net_connect.disconnect()
 
    except Exception as e:
        e = str(e)
        print(e)
 
thread(u=username, p=password)
 
filename = 'TestOutput.xlsx'
workbook = xlsxwriter.Workbook(filename)   # excel file
ws = workbook.add_worksheet('Results')     # Worksheet
 
headerformat = workbook.add_format({'bold': True,
                                    'font_name': 'Calibri',
                                    'font_size': 11,
                                    'bg_color': 'gray',
                                    'align': 'center',
                                    'valign': 'vcenter',
                                    'text_wrap': True})
normal_format = workbook.add_format({'text_wrap': True})
 
ws.set_column(0, 1, 30, normal_format)
ws.autofilter('A1:B1')
ws.freeze_panes(0, 1)
 
ws.write(0, 0, 'IP Address', headerformat)  # write column to the header
ws.write(0, 1, 'Result', headerformat)
 
 
# write data to the spreadsheet
row = 1 
for item in outputlist:
    item2 = item.split('###')
    ws.write(row, 0, item2[0])
    ws.write(row, 1, item2[1])
    row += 1
 
workbook.close()

